### This document analyzes the Rate My Hydrograph ratings.

In [1]:
%load_ext autoreload
%autoreload 2
import math
from collections import defaultdict
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import numpy as np
import itertools
import xarray
from pathlib import Path
from neuralhydrology.evaluation.metrics import kge, nse, fdc_fhv, fdc_flv

df = pd.read_csv('data/rmh-stage1.csv', index_col=0)
df.shape

/home/m/miniconda3/envs/maps/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(14586, 29)

The following tables show the percentage of times a model "won" or "lost" in a comparison, and the fractions of times it was rated as "equally good" or "equally bad".

In [2]:
def rank(df):
    stats = {}
    for model in df["model_a"].unique():
        # For every rating, exactly one of the num_* columns is 1.
        lw=df[(df["model_a"]==model) & (df["num_a_wins"] == 1)]
        ll=df[(df["model_a"]==model) & (df["num_b_wins"] == 1)]
        rev_lw=df[(df["model_b"]==model) & (df["num_b_wins"] == 1)]
        rev_ll=df[(df["model_b"]==model) & (df["num_a_wins"] == 1)]
        eq_good = df[((df["model_b"]==model) | (df["model_a"] == model)) & (df["num_equal_good"] == 1)]
        eq_bad = df[((df["model_b"]==model) | (df["model_a"] == model)) & (df["num_equal_bad"] == 1)]
        total = df[((df["model_b"]==model) | (df["model_a"] == model))].shape[0]
        stats[model] = {'won': lw.shape[0] + rev_lw.shape[0],
                        'lost': ll.shape[0] + rev_ll.shape[0],
                        'equal good': eq_good.shape[0] / total, 'equal bad': eq_bad.shape[0] / total,
                        'number of ratings': total}

    stats = pd.DataFrame(stats).T
    stats.index.name = 'model'
    stats['win%'] = 100 * stats['won'] / (stats['won'] + stats['lost'])
    return stats.sort_values(by='win%')

def modelname(model_in):
    # converts model names as used during experiments into how they are called in manuscript
    model_in = model_in.lower()
    if model_in == 'lbrm':
        model_out = 'lbrm-cc-lumped'
    elif model_in == 'ml-lstm':
        model_out = 'lstm-lumped'
    elif model_in == 'gr4j-raven-lp':
        model_out = 'gr4j-lumped'
    elif model_in == 'hmets-raven-lp':
        model_out = 'hmets-lumped'
    elif model_in == 'blended-raven-lp':
        model_out = 'blended-lumped'
    elif model_in == 'blended-raven-sd':
        model_out = 'blended-raven'
    elif model_in == 'hymod2':
        model_out = 'hymod2-lumped'
    else:
        model_out = model_in
    return model_out

In [3]:
rank_dfs = {}
for t in ["all combined", "overall", "high-flow", "low-flow"]:
    print(f'-----------------------------------------------------------------------\nStatistics for rating task "{t}"')
    t_df = df[df["task"]==t] if t != "all combined" else df.copy()
    rank_dfs[t] = rank(t_df)
    display(rank_dfs[t].style.format(precision=2).background_gradient(cmap='PiYG', subset=["win%"], vmin=0, vmax=100) \
        .highlight_null('white') \
        .background_gradient(cmap='Blues', subset=["equal good", "equal bad"], vmin=0, vmax=0.3))

-----------------------------------------------------------------------
Statistics for rating task "all combined"


,won,lost,equal good,equal bad,number of ratings,win%
model,,,,,,
MESH-CLASS-Raven,342.00,1076.00,0.08,0.28,2233.00,24.12
GEM-Hydro-Watroute,366.00,1063.00,0.10,0.27,2269.00,25.61
SWAT-Raven,469.00,951.00,0.11,0.26,2232.00,33.03
WATFLOOD-Raven,517.00,945.00,0.10,0.26,2292.00,35.36
MESH-SVS-Raven,526.00,905.00,0.11,0.25,2262.00,36.76
LBRM,656.00,699.00,0.16,0.24,2251.00,48.41
HYMOD2,673.00,671.00,0.17,0.22,2213.00,50.07
VIC-Raven,785.00,631.00,0.14,0.22,2228.00,55.44
HMETS-Raven-lp,773.00,564.00,0.17,0.23,2216.00,57.82


-----------------------------------------------------------------------
Statistics for rating task "overall"


,won,lost,equal good,equal bad,number of ratings,win%
model,,,,,,
MESH-CLASS-Raven,130.00,465.00,0.06,0.27,891.00,21.85
GEM-Hydro-Watroute,137.00,469.00,0.09,0.24,909.00,22.61
MESH-SVS-Raven,187.00,404.00,0.11,0.25,918.00,31.64
SWAT-Raven,192.00,394.00,0.10,0.23,881.00,32.76
WATFLOOD-Raven,217.00,390.00,0.09,0.25,927.00,35.75
LBRM,280.00,293.00,0.16,0.20,904.00,48.87
HYMOD2,286.00,256.00,0.17,0.20,858.00,52.77
VIC-Raven,322.00,250.00,0.13,0.21,864.00,56.29
HMETS-Raven-lp,344.00,249.00,0.14,0.21,911.00,58.01


-----------------------------------------------------------------------
Statistics for rating task "high-flow"


,won,lost,equal good,equal bad,number of ratings,win%
model,,,,,,
GEM-Hydro-Watroute,97.00,347.00,0.10,0.28,722.00,21.85
MESH-SVS-Raven,135.00,293.00,0.12,0.27,701.00,31.54
SWAT-Raven,132.00,284.00,0.12,0.28,695.00,31.73
MESH-CLASS-Raven,140.00,286.00,0.09,0.29,688.00,32.86
WATFLOOD-Raven,166.00,285.00,0.11,0.26,719.00,36.81
LBRM,216.00,188.00,0.15,0.28,707.00,53.47
HYMOD2,216.00,187.00,0.18,0.26,714.00,53.60
HMETS-Raven-lp,216.00,156.00,0.20,0.24,672.00,58.06
VIC-Raven,261.00,175.00,0.16,0.24,716.00,59.86


-----------------------------------------------------------------------
Statistics for rating task "low-flow"


,won,lost,equal good,equal bad,number of ratings,win%
model,,,,,,
MESH-CLASS-Raven,72.00,325.00,0.10,0.29,654.00,18.14
WATFLOOD-Raven,134.00,270.00,0.09,0.29,646.00,33.17
SWAT-Raven,145.00,273.00,0.10,0.26,656.00,34.69
GEM-Hydro-Watroute,132.00,247.00,0.10,0.30,638.00,34.83
LBRM,160.00,218.00,0.16,0.25,640.00,42.33
HYMOD2,171.00,228.00,0.15,0.22,641.00,42.86
VIC-Raven,202.00,206.00,0.13,0.24,648.00,49.51
MESH-SVS-Raven,204.00,208.00,0.12,0.24,643.00,49.51
HMETS-Raven-lp,213.00,159.00,0.17,0.24,633.00,57.26


In [4]:
grip_kges = {'mesh-class-raven': 0.45,
        'gem-hydro-watroute': 0.46,
        'mesh-svs-raven': 0.57,
        'swat-raven': 0.56,
        'watflood-raven': 0.62,
        'lbrm-cc-lumped': 0.75,
        'hymod2-lumped': 0.76,
        'vic-raven': 0.75,
        'hmets-lumped': 0.75,
        'blended-raven': 0.76,
        'blended-lumped': 0.79,
        'gr4j-lumped': 0.74,
        'lstm-lumped': 0.82}

paper_df = pd.concat({t: rank_df['win%'].rename(modelname, axis=0) for t, rank_df in rank_dfs.items() if t != 'all combined'}, axis=1)
model_order = paper_df['overall'].sort_values().index
paper_df['Median KGE from GRIP-GL'] = pd.Series(grip_kges)
paper_df = paper_df.reindex(model_order)
win_cols = list(rank_dfs.keys())[1:]
kge_col = paper_df.columns[-1]
latex = paper_df.style.format(precision=0, subset=win_cols).format(precision=2, subset=kge_col) \
       .background_gradient(cmap='PiYG', vmin=0, vmax=100, subset=win_cols) \
    .to_latex(convert_css=True, siunitx=True, hrules=True)
print(latex)

\begin{tabular}{lSSSS}
\toprule
{} & {overall} & {high-flow} & {low-flow} & {Median KGE from GRIP-GL} \\
{model} & {} & {} & {} & {} \\
\midrule
mesh-class-raven & {\cellcolor[HTML]{E181B5}} \color[HTML]{F1F1F1} 22 & {\cellcolor[HTML]{F5C2E0}} \color[HTML]{000000} 33 & {\cellcolor[HTML]{D965A4}} \color[HTML]{F1F1F1} 18 & 0.45 \\
gem-hydro-watroute & {\cellcolor[HTML]{E286B8}} \color[HTML]{F1F1F1} 23 & {\cellcolor[HTML]{E181B5}} \color[HTML]{F1F1F1} 22 & {\cellcolor[HTML]{F7CBE4}} \color[HTML]{000000} 35 & 0.46 \\
mesh-svs-raven & {\cellcolor[HTML]{F3BDDE}} \color[HTML]{000000} 32 & {\cellcolor[HTML]{F3BCDD}} \color[HTML]{000000} 32 & {\cellcolor[HTML]{F7F6F7}} \color[HTML]{000000} 50 & 0.57 \\
swat-raven & {\cellcolor[HTML]{F4C1DF}} \color[HTML]{000000} 33 & {\cellcolor[HTML]{F3BDDE}} \color[HTML]{000000} 32 & {\cellcolor[HTML]{F6C9E3}} \color[HTML]{000000} 35 & 0.56 \\
watflood-raven & {\cellcolor[HTML]{F8CEE6}} \color[HTML]{000000} 36 & {\cellcolor[HTML]{F9D3E8}} \color[HTML]{000000}

### Direct comparisons

We can also look at direct model comparisons. The following matrix can be read like this: Among the comparisons of model A in row i and model B in column j, the win-loss ratio of model A was `Matrix[i, j]`.

In [5]:
models = df['model_a'].unique()
matrix = defaultdict(dict)
for m1 in models:
    for m2 in models:
        sub_df = df[((df['model_a'] == m1) & (df['model_b'] == m2)) | ((df['model_b'] == m1) & (df['model_a'] == m2))]
        if len(sub_df) > 0:
            matrix[m1][m2] = rank(sub_df).loc[m1]['win%']
            matrix[m2][m1] = rank(sub_df).loc[m2]['win%']

compare_df = pd.DataFrame(matrix).T
compare_df = compare_df.loc[compare_df.columns]
compare_df = compare_df.rename(modelname, axis=0).rename(modelname, axis=1).reindex(model_order, axis=0).reindex(model_order, axis=1)
styled = compare_df.style.format(precision=0).background_gradient(cmap='PiYG', vmin=0, vmax=100).highlight_null(props='background-color: white; color: white')
display(styled)
print(styled.to_latex(convert_css=True, siunitx=True, hrules=True))

model,mesh-class-raven,gem-hydro-watroute,mesh-svs-raven,swat-raven,watflood-raven,lbrm-cc-lumped,hymod2-lumped,vic-raven,hmets-lumped,blended-raven,gr4j-lumped,blended-lumped,lstm-lumped
model,,,,,,,,,,,,,
mesh-class-raven,nan,50,31,35,37,20,23,22,22,22,16,21,7
gem-hydro-watroute,50,nan,35,43,35,21,33,25,18,24,16,16,5
mesh-svs-raven,69,65,nan,49,50,45,32,29,31,26,21,32,14
swat-raven,65,57,51,nan,47,28,31,30,29,24,19,23,7
watflood-raven,63,65,50,53,nan,33,34,34,26,29,17,25,15
lbrm-cc-lumped,80,79,55,72,67,nan,34,42,38,31,37,35,12
hymod2-lumped,77,67,68,69,66,66,nan,38,41,29,31,36,14
vic-raven,78,75,71,70,66,58,62,nan,55,41,35,37,16
hmets-lumped,78,82,69,71,74,62,59,45,nan,49,42,46,14


\begin{tabular}{lSSSSSSSSSSSSS}
\toprule
{model} & {mesh-class-raven} & {gem-hydro-watroute} & {mesh-svs-raven} & {swat-raven} & {watflood-raven} & {lbrm-cc-lumped} & {hymod2-lumped} & {vic-raven} & {hmets-lumped} & {blended-raven} & {gr4j-lumped} & {blended-lumped} & {lstm-lumped} \\
{model} & {} & {} & {} & {} & {} & {} & {} & {} & {} & {} & {} & {} & {} \\
\midrule
mesh-class-raven & {\cellcolor[HTML]{000000}} \color[HTML]{F1F1F1} {\cellcolor{white}} \color{white} nan & {\cellcolor[HTML]{F7F7F6}} \color[HTML]{000000} 50 & {\cellcolor[HTML]{F2BADC}} \color[HTML]{000000} 31 & {\cellcolor[HTML]{F7CBE4}} \color[HTML]{000000} 35 & {\cellcolor[HTML]{FAD4E9}} \color[HTML]{000000} 37 & {\cellcolor[HTML]{DF79B0}} \color[HTML]{F1F1F1} 20 & {\cellcolor[HTML]{E48BBC}} \color[HTML]{F1F1F1} 23 & {\cellcolor[HTML]{E283B7}} \color[HTML]{F1F1F1} 22 & {\cellcolor[HTML]{E283B7}} \color[HTML]{F1F1F1} 22 & {\cellcolor[HTML]{E286B8}} \color[HTML]{F1F1F1} 22 & {\cellcolor[HTML]{D34F99}} \color[HTML]{F1F1F

### Splitting by demographic data

In [6]:
prev_years = -np.inf
year_ranks = {}
participants = {}
for years in [5, 10, 15, 20, np.inf]:
    year_df = df[(df["years_experience"] < years) & (df["years_experience"] >= prev_years)]
    key = f"{prev_years} -- {years}"
    year_ranks[key] = rank(year_df)
    participants[key] = year_df["user_id"].nunique()
    print(f'< {years} years: Number of ratings: {year_df.shape[0]}, users: {participants[key]}')
    prev_years = years
year_ranks = pd.concat(year_ranks, axis=1).loc[:, (slice(None), "win%")].rename(modelname, axis=0).loc[model_order]
year_ranks.columns = year_ranks.columns.droplevel(1)
year_ranks = year_ranks.append(pd.Series(participants, name='Number of participants'))
styled = year_ranks.style.format(precision=0).background_gradient(cmap='PiYG', vmin=0, vmax=100, subset=(year_ranks.index[:-1], year_ranks.columns))
display(styled)

print(styled.to_latex(convert_css=True, siunitx=True, hrules=True))

< 5 years: Number of ratings: 3727, users: 168
< 10 years: Number of ratings: 3232, users: 151
< 15 years: Number of ratings: 2323, users: 121
< 20 years: Number of ratings: 1513, users: 67
< inf years: Number of ratings: 3791, users: 115


/tmp/ipykernel_18322/4293845357.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  year_ranks = year_ranks.append(pd.Series(participants, name='Number of participants'))


,-inf -- 5,5 -- 10,10 -- 15,15 -- 20,20 -- inf
model,,,,,
mesh-class-raven,22,28,28,21,21
gem-hydro-watroute,24,25,26,25,27
mesh-svs-raven,36,35,31,39,42
swat-raven,33,38,30,34,30
watflood-raven,35,35,32,34,39
lbrm-cc-lumped,46,49,43,52,52
hymod2-lumped,51,50,51,48,49
vic-raven,53,56,56,55,57
hmets-lumped,55,58,59,55,61


\begin{tabular}{lSSSSS}
\toprule
{} & {-inf -- 5} & {5 -- 10} & {10 -- 15} & {15 -- 20} & {20 -- inf} \\
{model} & {} & {} & {} & {} & {} \\
\midrule
mesh-class-raven & {\cellcolor[HTML]{E286B8}} \color[HTML]{F1F1F1} 22 & {\cellcolor[HTML]{EEABD2}} \color[HTML]{000000} 28 & {\cellcolor[HTML]{EDA8D1}} \color[HTML]{000000} 28 & {\cellcolor[HTML]{E07EB3}} \color[HTML]{F1F1F1} 21 & {\cellcolor[HTML]{E07EB3}} \color[HTML]{F1F1F1} 21 \\
gem-hydro-watroute & {\cellcolor[HTML]{E692C1}} \color[HTML]{000000} 24 & {\cellcolor[HTML]{E795C3}} \color[HTML]{000000} 25 & {\cellcolor[HTML]{EA9FCA}} \color[HTML]{000000} 26 & {\cellcolor[HTML]{E795C3}} \color[HTML]{000000} 25 & {\cellcolor[HTML]{EBA3CD}} \color[HTML]{000000} 27 \\
mesh-svs-raven & {\cellcolor[HTML]{F8D0E7}} \color[HTML]{000000} 36 & {\cellcolor[HTML]{F6C9E3}} \color[HTML]{000000} 35 & {\cellcolor[HTML]{F2B8DB}} \color[HTML]{000000} 31 & {\cellcolor[HTML]{FCDBED}} \color[HTML]{000000} 39 & {\cellcolor[HTML]{FCE5F1}} \color[HTML]{000000} 4

In [7]:
area_rank_dfs = {}
participants = {}
for t in ["all combined", "overall", "high-flow","low-flow"]:
    print(f'-----------------------------------------------------------------------\nStatistics for rating task "{t}"')
    t_df = df[df["task"]==t] if t != "all combined" else df.copy()
    focus_df = pd.concat([t_df["focus_areas"].str.split(',').apply(lambda x: pd.Series(x, index=[f"focus-{i}" for i in range(len(x))])), t_df], axis=1)
    index = ['flood-modeling', 'water-resources', 'drought-modeling', 'waterquality', 'hydropower', 'consulting', 'social-sciences', 'other']
    labels = ['Flood modeling', 'Water resources', 'Drought modeling', 'Water quality', 'Hydropower', 'Consulting', 'Social sciences', 'Other']
    area_ranks = {}
    for label, area in zip(labels, index):
        area_df = focus_df.loc[(focus_df[[c for c in focus_df.columns if c.startswith("focus-")]] == area).any(axis=1)].copy()
        area_ranks[area] = rank(area_df)
        print(f'{area}: Number of ratings: {area_df.shape[0]}, users: {area_df["user_id"].nunique()}')

    area_df = focus_df.loc[((focus_df[[c for c in focus_df.columns if c.startswith("focus-")]] == "flood-modeling").any(axis=1)) \
        & (focus_df[[c for c in focus_df.columns if c.startswith("focus-")]] != "drought-modeling").all(axis=1)].copy()
    area = "flood but not drought modeling"
    area_ranks[area] = rank(area_df)
    print(f'{area}: Number of ratings: {area_df.shape[0]}, users: {area_df["user_id"].nunique()}')
    participants[(t, area)] = area_df["user_id"].nunique()

    area_df = focus_df.loc[((focus_df[[c for c in focus_df.columns if c.startswith("focus-")]] == "drought-modeling").any(axis=1)) \
        & (focus_df[[c for c in focus_df.columns if c.startswith("focus-")]] != "flood-modeling").all(axis=1)].copy()
    area = "drought but not flood modeling"
    area_ranks[area] = rank(area_df)
    print(f'{area}: Number of ratings: {area_df.shape[0]}, users: {area_df["user_id"].nunique()}')

    area_ranks = pd.concat(area_ranks, axis=1).loc[:, (slice(None), "win%")]
    area_rank_dfs[t] = area_ranks
    participants[(t, area)] = area_df["user_id"].nunique()
    display(area_ranks.sort_values(by=area_ranks.columns[0]).style.format(precision=2).background_gradient(cmap='PiYG', vmin=0, vmax=100))

-----------------------------------------------------------------------
Statistics for rating task "all combined"
flood-modeling: Number of ratings: 9177, users: 348
water-resources: Number of ratings: 7233, users: 301
drought-modeling: Number of ratings: 3467, users: 133
waterquality: Number of ratings: 1881, users: 94
hydropower: Number of ratings: 1974, users: 83
consulting: Number of ratings: 2013, users: 59
social-sciences: Number of ratings: 310, users: 13
other: Number of ratings: 1566, users: 80
flood but not drought modeling: Number of ratings: 6594, users: 253
drought but not flood modeling: Number of ratings: 884, users: 38


,flood-modeling,water-resources,drought-modeling,waterquality,hydropower,consulting,social-sciences,other,flood but not drought modeling,drought but not flood modeling
,win%,win%,win%,win%,win%,win%,win%,win%,win%,win%
model,,,,,,,,,,
GEM-Hydro-Watroute,23.60,24.72,25.07,23.89,20.27,25.47,25.00,28.48,23.79,30.43
MESH-CLASS-Raven,24.44,22.08,23.95,27.47,18.52,21.18,33.33,21.28,25.91,34.57
SWAT-Raven,31.75,32.35,31.87,28.65,38.86,37.22,25.00,33.76,32.77,37.61
WATFLOOD-Raven,35.96,36.38,39.02,34.92,36.63,34.08,24.14,34.12,35.49,43.00
MESH-SVS-Raven,37.75,36.60,38.28,30.32,41.03,43.59,35.48,37.09,37.48,37.86
LBRM,48.26,51.15,46.63,43.86,52.85,46.88,54.05,48.44,48.09,41.84
HYMOD2,50.86,48.86,47.65,41.57,45.26,45.58,50.00,46.75,52.34,48.48
VIC-Raven,55.53,55.56,55.93,55.62,55.14,54.55,52.78,52.00,55.35,55.66


-----------------------------------------------------------------------
Statistics for rating task "overall"
flood-modeling: Number of ratings: 3558, users: 348
water-resources: Number of ratings: 2880, users: 301
drought-modeling: Number of ratings: 1354, users: 133
waterquality: Number of ratings: 760, users: 94
hydropower: Number of ratings: 782, users: 83
consulting: Number of ratings: 757, users: 59
social-sciences: Number of ratings: 120, users: 13
other: Number of ratings: 678, users: 80
flood but not drought modeling: Number of ratings: 2563, users: 253
drought but not flood modeling: Number of ratings: 359, users: 38


,flood-modeling,water-resources,drought-modeling,waterquality,hydropower,consulting,social-sciences,other,flood but not drought modeling,drought but not flood modeling
,win%,win%,win%,win%,win%,win%,win%,win%,win%,win%
model,,,,,,,,,,
GEM-Hydro-Watroute,20.72,19.22,21.90,18.99,12.50,18.18,15.38,29.11,20.85,26.47
MESH-CLASS-Raven,21.91,20.27,19.38,26.74,14.89,17.65,41.18,17.74,23.57,25.00
MESH-SVS-Raven,30.59,31.60,35.37,25.64,38.71,35.29,36.36,39.19,28.24,32.65
SWAT-Raven,31.96,32.74,29.05,24.69,35.71,43.06,25.00,31.88,34.71,36.73
WATFLOOD-Raven,38.64,35.89,37.76,35.53,42.31,38.55,38.46,27.78,39.77,42.00
LBRM,51.08,53.18,49.65,44.59,56.10,51.79,33.33,41.67,49.12,36.96
HYMOD2,53.15,50.21,46.51,47.30,45.45,43.48,60.00,50.00,56.84,53.33
VIC-Raven,56.32,56.03,57.48,51.32,53.12,52.05,57.14,58.49,55.00,51.28


-----------------------------------------------------------------------
Statistics for rating task "high-flow"
flood-modeling: Number of ratings: 2917, users: 304
water-resources: Number of ratings: 2285, users: 258
drought-modeling: Number of ratings: 1091, users: 114
waterquality: Number of ratings: 592, users: 77
hydropower: Number of ratings: 605, users: 67
consulting: Number of ratings: 644, users: 47
social-sciences: Number of ratings: 95, users: 13
other: Number of ratings: 464, users: 62
flood but not drought modeling: Number of ratings: 2104, users: 223
drought but not flood modeling: Number of ratings: 278, users: 33


,flood-modeling,water-resources,drought-modeling,waterquality,hydropower,consulting,social-sciences,other,flood but not drought modeling,drought but not flood modeling
,win%,win%,win%,win%,win%,win%,win%,win%,win%,win%
model,,,,,,,,,,
GEM-Hydro-Watroute,21.77,21.98,18.69,11.11,15.38,27.59,20.00,27.91,23.32,20.69
SWAT-Raven,30.42,31.58,29.73,25.53,40.74,29.63,37.50,31.25,32.08,36.67
MESH-SVS-Raven,31.15,31.22,26.88,26.98,30.23,42.11,12.50,30.95,34.04,38.10
MESH-CLASS-Raven,31.34,28.57,34.74,38.00,33.33,29.41,25.00,33.33,32.70,59.09
WATFLOOD-Raven,34.43,39.25,41.05,36.84,35.29,30.61,20.00,41.07,33.66,51.85
LBRM,49.79,57.50,57.65,44.00,50.91,46.51,75.00,59.46,48.33,65.38
HYMOD2,55.19,53.16,50.49,42.00,48.08,51.02,40.00,45.24,56.98,50.00
HMETS-Raven-lp,57.52,65.19,51.16,59.62,70.37,76.09,62.50,53.85,56.65,36.36


-----------------------------------------------------------------------
Statistics for rating task "low-flow"
flood-modeling: Number of ratings: 2702, users: 292
water-resources: Number of ratings: 2068, users: 239
drought-modeling: Number of ratings: 1022, users: 110
waterquality: Number of ratings: 529, users: 72
hydropower: Number of ratings: 587, users: 65
consulting: Number of ratings: 612, users: 45
social-sciences: Number of ratings: 95, users: 13
other: Number of ratings: 424, users: 62
flood but not drought modeling: Number of ratings: 1927, users: 212
drought but not flood modeling: Number of ratings: 247, users: 30


,flood-modeling,water-resources,drought-modeling,waterquality,hydropower,consulting,social-sciences,other,flood but not drought modeling,drought but not flood modeling
,win%,win%,win%,win%,win%,win%,win%,win%,win%,win%
model,,,,,,,,,,
MESH-CLASS-Raven,20.16,18.04,18.82,17.39,8.47,17.65,28.57,13.51,21.51,26.09
GEM-Hydro-Watroute,30.49,35.68,35.92,41.07,35.94,35.14,44.44,27.91,29.53,44.83
SWAT-Raven,32.70,32.63,38.10,38.00,40.58,37.04,12.50,40.00,30.85,40.00
WATFLOOD-Raven,33.86,33.86,38.89,32.14,30.36,29.79,9.09,35.71,31.52,34.78
LBRM,42.98,41.30,32.65,42.55,50.00,42.62,60.00,48.39,46.47,26.92
HYMOD2,43.44,42.70,46.30,33.33,42.62,42.31,44.44,43.18,41.52,42.86
VIC-Raven,49.61,48.89,44.94,56.25,46.94,55.77,30.00,40.00,52.02,51.72
MESH-SVS-Raven,54.84,48.58,53.61,42.55,52.54,52.86,50.00,40.00,53.80,45.45


In [8]:
flood = 'flood but not drought modeling'
drought = 'drought but not flood modeling'
tasks = ['overall', 'high-flow', 'low-flow']
paper_df = pd.concat({t: ardf.loc[:, [(flood, 'win%'), (drought, 'win%')]].rename(modelname, axis=0) for t, ardf in area_rank_dfs.items() if t != 'all combined'}, axis=1)
paper_df.columns = paper_df.columns.droplevel(2).reorder_levels([1, 0])
styled = paper_df.loc[model_order, [(flood, t) for t in tasks] + [(drought, t) for t in tasks]].style.format(precision=0).background_gradient(cmap='PiYG', vmin=0, vmax=100)
display(styled)
display(pd.Series(participants))

print(styled.to_latex(convert_css=True, siunitx=True, hrules=True))

all combined  flood but not drought modeling    253
              drought but not flood modeling     38
overall       flood but not drought modeling    253
              drought but not flood modeling     38
high-flow     flood but not drought modeling    223
              drought but not flood modeling     33
low-flow      flood but not drought modeling    212
              drought but not flood modeling     30
dtype: int64

\begin{tabular}{lSSSSSS}
\toprule
{} & \multicolumn{3}{r}{flood but not drought modeling} & \multicolumn{3}{r}{drought but not flood modeling} \\
{} & {overall} & {high-flow} & {low-flow} & {overall} & {high-flow} & {low-flow} \\
{model} & {} & {} & {} & {} & {} & {} \\
\midrule
mesh-class-raven & {\cellcolor[HTML]{E58DBE}} \color[HTML]{F1F1F1} 24 & {\cellcolor[HTML]{F4C1DF}} \color[HTML]{000000} 33 & {\cellcolor[HTML]{E181B5}} \color[HTML]{F1F1F1} 22 & {\cellcolor[HTML]{E897C4}} \color[HTML]{000000} 25 & {\cellcolor[HTML]{E7F5D3}} \color[HTML]{000000} 59 & {\cellcolor[HTML]{E99CC8}} \color[HTML]{000000} 26 \\
gem-hydro-watroute & {\cellcolor[HTML]{DF7CB1}} \color[HTML]{F1F1F1} 21 & {\cellcolor[HTML]{E48BBC}} \color[HTML]{F1F1F1} 23 & {\cellcolor[HTML]{F0B2D7}} \color[HTML]{000000} 30 & {\cellcolor[HTML]{EA9FCA}} \color[HTML]{000000} 26 & {\cellcolor[HTML]{DF79B0}} \color[HTML]{F1F1F1} 21 & {\cellcolor[HTML]{FAEBF3}} \color[HTML]{000000} 45 \\
mesh-svs-raven & {\cellcolor[HTML]{EEABD2}

In [9]:
area_ranks = {}
for area in df["occupation"].unique():
    area_df = df[df["occupation"] == area]
    area_ranks[area] = rank(area_df)
    print(f'{area}: Number of ratings: {area_df.shape[0]}, users: {area_df["user_id"].nunique()}')
area_ranks = pd.concat(area_ranks, axis=1).loc[:, (slice(None), "win%")].rename(modelname, axis=0)
area_ranks.columns = area_ranks.columns.droplevel(1)
styled = area_ranks.loc[model_order, ['academia', 'public-sector', 'industry']].style.format(precision=0).background_gradient(cmap='PiYG', vmin=0, vmax=100)
display(styled)

print(styled.to_latex(convert_css=True, siunitx=True, hrules=True))

public-sector: Number of ratings: 3122, users: 122
academia: Number of ratings: 9838, users: 408
industry: Number of ratings: 1626, users: 92


,academia,public-sector,industry
model,,,
mesh-class-raven,25,23,20
gem-hydro-watroute,26,23,29
mesh-svs-raven,35,41,40
swat-raven,32,33,38
watflood-raven,37,34,29
lbrm-cc-lumped,47,50,51
hymod2-lumped,49,52,54
vic-raven,57,54,50
hmets-lumped,58,56,61


\begin{tabular}{lSSS}
\toprule
{} & {academia} & {public-sector} & {industry} \\
{model} & {} & {} & {} \\
\midrule
mesh-class-raven & {\cellcolor[HTML]{E897C4}} \color[HTML]{000000} 25 & {\cellcolor[HTML]{E388BA}} \color[HTML]{F1F1F1} 23 & {\cellcolor[HTML]{DE77AE}} \color[HTML]{F1F1F1} 20 \\
gem-hydro-watroute & {\cellcolor[HTML]{E99CC8}} \color[HTML]{000000} 26 & {\cellcolor[HTML]{E388BA}} \color[HTML]{F1F1F1} 23 & {\cellcolor[HTML]{EEADD4}} \color[HTML]{000000} 29 \\
mesh-svs-raven & {\cellcolor[HTML]{F6C9E3}} \color[HTML]{000000} 35 & {\cellcolor[HTML]{FDE2F0}} \color[HTML]{000000} 41 & {\cellcolor[HTML]{FDE1EF}} \color[HTML]{000000} 40 \\
swat-raven & {\cellcolor[HTML]{F3BDDE}} \color[HTML]{000000} 32 & {\cellcolor[HTML]{F5C4E1}} \color[HTML]{000000} 33 & {\cellcolor[HTML]{FBD9EC}} \color[HTML]{000000} 38 \\
watflood-raven & {\cellcolor[HTML]{F9D3E8}} \color[HTML]{000000} 37 & {\cellcolor[HTML]{F6C9E3}} \color[HTML]{000000} 34 & {\cellcolor[HTML]{EEADD4}} \color[HTML]{000000} 29 